In [1]:
import cv2
import pyautogui
import time
import numpy as np
from matplotlib import pyplot as plt

pyautogui.FAILSAFE = True

In [45]:
class Mahjong:
    def __init__(self):
        threshold = 0.2
        self.min_th, self.max_th = (1-threshold, 1+threshold)
        self.img_path = '/home/jakub/Pictures/mahjong.png'
        self.pattern_path = '/home/jakub/Pictures/pattern.png'
        self.rec_ratio, self.height_ratio, self.i_w, self.i_h = self.get_ratios()
        
    def get_ratios(self):
        img_rgb = cv2.imread(self.img_path, 0)
        i_w, i_h = img_rgb.shape[::-1]
        template = cv2.imread(self.pattern_path, 0)
        t_w, t_h = template.shape[::-1]
        rec_ratio = t_w/t_h  # Expected tile ratio
        height_ratio = t_h/i_h  # Expected tile-to-board height ratio
        return rec_ratio, height_ratio, i_w, i_h
    
    def capture_screenshot(self):
        image = pyautogui.screenshot()  # Captured image in PIL
        cv_image = np.array(image)  # Cv2 compatible image
        return cv_image[:, :, ::-1]  # Convert RGB to BGR 
    
    def loc_tiles(self):
        self.tiles = []
        self.lonely = []
        self.screen = self.capture_screenshot().copy()
        img_gray = cv2.cvtColor(self.screen, cv2.COLOR_BGR2GRAY)
        binary = cv2.bitwise_not(img_gray)
        (contours,_) = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
        
        for contour in contours:
            (x,y,w,h) = cv2.boundingRect(contour)
            if (w/h > (self.rec_ratio * self.min_th)
                and w/h < (self.rec_ratio * self.max_th)
                and h/self.i_h > (self.height_ratio * self.min_th)
                and h/self.i_h < (self.height_ratio * self.max_th)):
                self.tiles.append((x, y, h, w))
        print('%d tiles detected.' % len(self.tiles))
    
    def loc_siblings(self, i):
        x, y, h, w = self.tiles[i]
        look_for = self.screen[y:y+h, x:x+w]
        res = cv2.matchTemplate(self.screen, look_for, cv2.TM_CCOEFF_NORMED)
        threshold = 0.8
        loc = np.where(res >= threshold)
        look = zip(*loc[::-1])
        for pt in look:
            if loc[::-1][0].size == 1:
                print('im lonely')
                self.lonely.append(pt)
                break
            pyautogui.click(x=(pt[0]+w/2), y=(pt[1]+h/2))

    def solve(self):
        start = time.time()
        self.loc_tiles()
        for tile in range(len(self.tiles)):
            self.loc_siblings(tile)
        print(self.lonely)
        stop = time.time()
        return stop - start

In [46]:
m = Mahjong()

In [47]:
m.solve()

86 tiles detected.
im lonely
im lonely
im lonely
im lonely
im lonely
im lonely
im lonely
im lonely


FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.

In [4]:
import mss
s = mss.mss()
s.monitors

[{'left': 0, 'top': 0, 'width': 1920, 'height': 1080},
 {'left': 0, 'top': 0, 'width': 1920, 'height': 1080}]

In [5]:
image = pyautogui.screenshot()  # Captured image in PIL
cv_image = np.array(image)  # Cv2 compatible image
i = cv_image[:, :, ::-1]  # Convert RGB to BGR 

In [6]:
threshold = 0.2
min_th, max_th = (1-threshold, 1+threshold)
img_path = '/home/jakub/Pictures/mahjong.png'
pattern_path = '/home/jakub/Pictures/pattern.png'

In [7]:
def get_ratios():
    img_rgb = cv2.imread(img_path, 0)
    i_w, i_h = img_rgb.shape[::-1]
    template = cv2.imread(pattern_path, 0)
    t_w, t_h = template.shape[::-1]
    rec_ratio = t_w/t_h  # Expected tile ratio
    height_ratio = t_h/i_h  # Expected tile-to-board height ratio
    return rec_ratio, height_ratio, i_w, i_h

rec_ratio, height_ratio, i_w, i_h = get_ratios()

In [8]:
def loc_tiles(screen):
    tiles = []
    lonely = []
    img_gray = cv2.cvtColor(screen, cv2.COLOR_BGR2GRAY)
    binary = cv2.bitwise_not(img_gray)
    (contours,_) = cv2.findContours(binary, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

    for contour in contours:
        (x,y,w,h) = cv2.boundingRect(contour)
        if (w/h > (rec_ratio * min_th)
            and w/h < (rec_ratio * max_th)
            and h/i_h > (height_ratio * min_th)
            and h/i_h < (height_ratio * max_th)):
            tiles.append((x, y, h, w))
    print('%d tiles detected.' % len(tiles))
    return tiles

In [12]:
def try_tile(screen, tile):
    x, y, w, h = tile
    pre_click = screen[y:y+h, x:x+w]
    pyautogui.click(x=(x+w/2), y=(y+h/2))
    post_click = screen[y:y+h, x:x+w]
    difference = cv2.subtract(pre_click, post_click)
    b, g, r = cv2.split(difference)
    if cv2.countNonZero(b) == 0 and cv2.countNonZero(g) == 0 and cv2.countNonZero(r) == 0:
        print("Not clickable")
    else:
        print("Clickable")

In [10]:
loc_tiles(i)

88 tiles detected.


[(672, 562, 57, 39),
 (632, 562, 57, 39),
 (592, 562, 57, 39),
 (552, 562, 57, 39),
 (512, 562, 57, 39),
 (472, 562, 57, 39),
 (432, 562, 57, 39),
 (392, 562, 57, 39),
 (352, 562, 57, 39),
 (312, 562, 57, 39),
 (272, 562, 57, 39),
 (232, 562, 57, 39),
 (594, 504, 57, 37),
 (312, 504, 57, 39),
 (558, 498, 57, 39),
 (518, 498, 57, 39),
 (478, 498, 57, 39),
 (438, 498, 57, 39),
 (398, 498, 57, 39),
 (358, 498, 57, 39),
 (632, 447, 57, 39),
 (597, 447, 57, 34),
 (312, 447, 57, 39),
 (272, 447, 57, 39),
 (559, 441, 57, 38),
 (358, 441, 57, 39),
 (524, 435, 57, 39),
 (484, 435, 57, 39),
 (444, 435, 57, 39),
 (404, 435, 57, 39),
 (672, 389, 57, 39),
 (632, 389, 57, 39),
 (597, 389, 57, 34),
 (312, 389, 57, 39),
 (272, 389, 57, 39),
 (232, 389, 57, 39),
 (563, 383, 57, 34),
 (358, 383, 57, 39),
 (526, 377, 57, 37),
 (404, 377, 57, 39),
 (490, 371, 57, 39),
 (450, 371, 57, 39),
 (752, 360, 57, 39),
 (712, 360, 57, 39),
 (192, 360, 57, 39),
 (476, 336, 57, 39),
 (672, 331, 57, 39),
 (632, 331, 5

In [25]:
import time

import cv2
import mss
import numpy


with mss.mss() as sct:
    # Part of the screen to capture
    monitor = {"top": 0, "left": 0, "width": 960, "height": 1080}

    while "Screen capturing":
        last_time = time.time()

        # Get raw pixels from the screen, save it to a Numpy array
        img = numpy.array(sct.grab(monitor))
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)

        # Display the picture
        cv2.imshow("OpenCV/Numpy normal", img)
        tiles = loc_tiles(img)
        for tile in tiles:
            
            
            clickable(tile):
                x, y, w, h = tile
                
                img = numpy.array(sct.grab(monitor))
                img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
                pre_click = img[y:y+h, x:x+w]
                pyautogui.click(x=(x+w/2), y=(y+h/2))
                img = numpy.array(sct.grab(monitor))
                img = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
                post_click = img[y:y+h, x:x+w]
                difference = cv2.subtract(pre_click, post_click)
                b, g, r = cv2.split(difference)
                if cv2.countNonZero(b) == 0 and cv2.countNonZero(g) == 0 and cv2.countNonZero(r) == 0:
                    pass
                else:
                    return tile
        
        # Display the picture in grayscale
        # cv2.imshow('OpenCV/Numpy grayscale',
        #            cv2.cvtColor(img, cv2.COLOR_BGRA2GRAY))

        print("fps: {}".format(1 / (time.time() - last_time)))

        # Press "q" to quit
        if cv2.waitKey(25) & 0xFF == ord("q"):
            cv2.destroyAllWindows()
            break

87 tiles detected.
Clickable
Not clickable
Not clickable
Not clickable
Not clickable
Not clickable
Not clickable
Not clickable
Not clickable
Not clickable
Not clickable
Clickable
Clickable
Clickable
Not clickable
Not clickable
Not clickable
Not clickable
Clickable
Clickable
Not clickable
Not clickable
Clickable
Clickable
Clickable
Clickable
Not clickable
Not clickable
Clickable
Not clickable
Not clickable
Not clickable
Not clickable
Not clickable
Not clickable
Clickable
Clickable
Clickable
Clickable
Not clickable
Clickable
Clickable
Not clickable
Clickable
Clickable
Not clickable
Not clickable
Not clickable
Not clickable
Not clickable
Not clickable
Clickable
Clickable
Clickable
Clickable
Not clickable
Not clickable
Clickable
Not clickable
Not clickable
Clickable
Clickable
Clickable
Clickable
Not clickable
Not clickable
Clickable


FailSafeException: PyAutoGUI fail-safe triggered from mouse moving to a corner of the screen. To disable this fail-safe, set pyautogui.FAILSAFE to False. DISABLING FAIL-SAFE IS NOT RECOMMENDED.